In [1]:
import requests
import pandas
from bs4 import BeautifulSoup

In [69]:
# input
account = '106010'
password = 'G122117609'
character = 'self' # options: 'cloud', 'self'
ticket_type = 'all' # options: 'advanced', 'assigned', 'processing', 'finished', 'special', 'all'
add_file_url = True
join_ticket_detail = True

In [3]:
if character == 'self':
    character_id = account
elif character == 'cloud':
    character_id = '20'

In [4]:
def login(account, password):
    url = 'http://202.3.168.17:8080/login_check.jsp'
    data = {
        'sess_id': account,
        'sess_password': password,
        'dfForm': 'login.jsp',
    }
    session = requests.session()
    response = session.post(url=url, data=data)
    return session

In [5]:
session = login(account=account, password=password)

In [6]:
def request_dashboard(session, character_id):
    url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal.jsp?action=search'
    data = {
        'character': character_id,
    }
    response = session.post(url=url, data=data)
    return response

In [7]:
response_dashboard = request_dashboard(session=session, character_id=character_id)

In [49]:
def produce_ticket_tables(html):
    dataframes = pandas.read_html(html)
    # take the part we want to build a dictionary
    ticket_tables = {
        'advanced': dataframes[2],
        'assigned': dataframes[3],
        'processing': dataframes[4],
        'finished': dataframes[5],
        'special': dataframes[6],
    }
    return ticket_tables

In [50]:
ticket_tables = produce_ticket_tables(html=response_dashboard.text)

In [51]:
def use_first_row_as_title(dataframe):
    title = dataframe.iloc[0]
    dataframe = dataframe[1:]
    dataframe.columns = title
    return dataframe

In [52]:
def clean_data(dataframe):
    for index, row in dataframe.iterrows():
        row['單號'] = row['單號'].replace(' (Delay)', '')
    return dataframe

In [53]:
ticket_tables = {key:clean_data(use_first_row_as_title(value)) for key, value in ticket_tables.items()}

In [54]:
def add_column_file_url(dataframe, session):
    file_urls = []
    for index, row in dataframe.iterrows():
        file_url = ''
        try:
            ticket_number = row['單號']
            url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal_Detail.jsp'
            data = {
                'seq_no': ticket_number,
            }
            response = session.post(url=url, data=data)
            soup = BeautifulSoup(response.text, 'lxml')
            tags = soup.select('body > table > tr:nth-child(2) > td > fieldset > table > tr:nth-child(1) > td > ol > li > input[type=button]:nth-child(1)')
            file_url = tags[0]['onclick'].split(',')[0].split('"')[1]
        except:
            pass
        file_urls.append(file_url)
    dataframe = dataframe.assign(建置單檔案=file_urls)
    return dataframe

In [55]:
if add_file_url and ticket_type != 'all':
    ticket_tables[ticket_type] = add_column_file_url(ticket_tables[ticket_type], session=session)
elif add_file_url and ticket_type == 'all':
    ticket_tables = {key:add_column_file_url(value, session=session) for key, value in ticket_tables.items()}
else:
    pass

In [74]:
def produce_ticket_detail_table(session, character_id, dataframe):
    ticket_detail_dataframes = []
    for index, row in dataframe.iterrows():
        try:
            ticket_number = row['單號']
            url = 'http://202.3.168.17:8080/Disp/retriveDetail.jsp'
            data = {
                'method': 'get_Disp_DetailCons',
                'Disp_Cons_Seq': character_id,
                'Disp_Grp_Seq': character_id,
                'Disp_Seq': ticket_number,
            }
            detail_response = session.post(url=url, data=data)
            ticket_detail_dataframe = pandas.read_html(detail_response.text)[0]
            ticket_detail_dataframe = use_first_row_as_title(ticket_detail_dataframe)
            ticket_detail_dataframe = ticket_detail_dataframe.assign(單號=ticket_number)
            ticket_detail_dataframes.append(ticket_detail_dataframe)
        except:
            pass
    if ticket_detail_dataframes:
        ticket_detail_table = pandas.concat(ticket_detail_dataframes)
    else:
        ticket_detail_table = None
    return ticket_detail_table

In [76]:
if join_ticket_detail and ticket_type != 'all':
    ticket_detail_tables[ticket_type] = produce_ticket_detail_table(session=session, character_id=character_id, dataframe=ticket_tables[ticket_type])
elif join_ticket_detail and ticket_type == 'all':
    ticket_detail_tables = {key:produce_ticket_detail_table(session=session, character_id=character_id, dataframe=value) for key, value in ticket_tables.items()}
else:
    pass

In [79]:
ticket_detail_tables['assigned']

In [16]:
df_list = []
for ticket_number in list(ticket_tables[ticket_type]['單號']):
    ticket_detail_response = get_ticket_detail_with_given_ticket_number(session=session, chracter=chracters[chracter_choose], ticket_number=ticket_number)
    df_ticket_detail = get_dataframe_list_by_reading_html(ticket_detail_response.text)[0]
    df_ticket_detail = let_the_first_row_be_column_title(df_ticket_detail)
    df_ticket_detail = df_ticket_detail.assign(單號=ticket_number)
    df_list += [df_ticket_detail]

NameError: name 'chracters' is not defined

In [ ]:
df_left = pandas.concat(df_list)
df_right = ticket_tables[ticket_type]
db_joined = pandas.merge(df_left, df_right, on='單號', how='outer', suffixes=('', '-細項'))

In [23]:
db_joined['建置單檔案']

KeyError: '建置單檔案'